In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import cross_validate, GridSearchCV, KFold

In [ ]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 75)

# Read and clean data

In [ ]:
df = pd.read_csv('../../Data/ALSdatacleaned.csv',index_col=[0])

## Clean variables

**ALSFRS**

In [ ]:
# multiply alsfrs by 1.2
df['alsfrs'] = df['alsfrs']*1.2
df['alsfrsr'] = df[['alsfrs','alsfrsr']].fillna(method='ffill',axis=1)['alsfrsr']

In [ ]:
df.loc[df['alsfrsr']>48,'alsfrsr'] = 48

**FVC**  
Most subjects are Caucasian, so use formula for Caucasians to compute standard/normal FVC value using NHANESIII  
https://www.atsjournals.org/doi/10.1164/ajrccm.159.1.9712108?url_ver=Z39.88-2003&rfr_id=ori:rid:crossref.org&rfr_dat=cr_pub%20%200pubmed

In [ ]:
def fvc_normal(age,height,sex):
    if sex == 1:
        return -0.1933 + (0.00064*age) + (-0.000269*(age**2)) + (0.00018642*(height**2))
    elif sex == 0:
        return -0.3560 + (0.01870*age) + (-0.000382*(age**2)) + (0.00014815*(height**2))

In [ ]:
df['FVC_norm'] = df.apply(lambda x: fvc_normal(x.Age, x.Height, x.Sex), axis=1)

In [ ]:
df['FVC_perc_new'] = df['FVC_abs']/df['FVC_norm']
df['FVC_perc_new'] = df[['FVC_perc','FVC_perc_new']].fillna(method='ffill', axis=1)['FVC_perc_new']

**Others**

In [ ]:
df['delta'] = df['delta'].astype(int)

In [ ]:
df = df.dropna(subset=['Onset_Delta'])

**Drop unwanted columns and rows**

In [ ]:
# selected required columns
df = df[['subject_id', 'delta', 'Age', 'Sex', 'site_bulbar',
       'site_limb', 'Onset_Delta', 'Diagnosis_Delta', 'RiluzoleUse', 'Height',
       'Weight', 'BMI', 'Pulse', 'Respiratory_Rate', 'BP_Diastolic', 'BP_Systolic',
       'FVC_perc_new', 'ALT', 'AST', 'UricAcid', 'BUN',
       'Albumin', 'AbsNeutroCount', 'Protein', 'CK', 'TotCholesterol',
       'Triglycerides', 'HbA1c', 'Hb', 'Hematocrit', 'WBC', 'RBC',
       'Creatinine', 'Sodium', 'Potassium', 'Chloride', 'Glucose', 'Platelets',
       'AbsEosinophil', 'AlkalinePhosphatase', 'Bicarbonate', 'Calcium',
       'AbsLymphocyte', 'AbsMonocyte', 'AbsBasophil', 'BilirubinTotal', 'GGT', 'PercLymphocytes',
       'PercMonocytes', 'PercBasophils', 'Phosphorus', 'PercEosinophils', 'alsfrsr' ]]

In [ ]:
# drop subjects with more than 50% of labs missing across all time
df_tmp = df.groupby('subject_id').agg(np.nanmean)
lab_columns = ['ALT', 'AST',
       'UricAcid', 'BUN', 'Albumin', 'AbsNeutroCount', 'Protein', 'CK',
       'TotCholesterol', 'Triglycerides', 'HbA1c', 'Hb', 'Hematocrit', 'WBC',
       'RBC', 'Creatinine', 'Sodium', 'Potassium', 'Chloride', 'Glucose',
       'Platelets', 'AbsEosinophil', 'AlkalinePhosphatase', 'Bicarbonate',
       'Calcium', 'AbsLymphocyte', 'AbsMonocyte', 'AbsBasophil',
       'BilirubinTotal', 'GGT', 'PercLymphocytes', 'PercMonocytes',
       'PercBasophils', 'Phosphorus', 'PercEosinophils']
df = df[~df['subject_id'].isin(df_tmp[df_tmp[lab_columns].isna().sum(axis=1)>18].index)]

In [ ]:
df.to_csv('ALSdatacleaned_noimputation.csv')

## Data imputation  
Linear fit for missing data within subject. No missing data imputation for data that is missing across all timepoints for now.

In [ ]:
df = pd.read_csv('../Data/ALSdatacleaned_noimputation.csv', index_col=[0])

In [ ]:
# Data imputation for dynamic variables only, excluding BMI because BMI will be recomputed after weight imputation
imputed_vars = ['Weight', 'Pulse', 'Respiratory_Rate', 'BP_Diastolic', 'BP_Systolic', 'FVC_perc_new', 
                'ALT', 'AST', 'UricAcid', 'BUN', 'Albumin', 'AbsNeutroCount', 'Protein', 'CK',
                'TotCholesterol', 'Triglycerides', 'HbA1c', 'Hb', 'Hematocrit', 'WBC',
                'RBC', 'Creatinine', 'Sodium', 'Potassium', 'Chloride', 'Glucose',
                'Platelets', 'AbsEosinophil', 'AlkalinePhosphatase', 'Bicarbonate',
                'Calcium', 'AbsLymphocyte', 'AbsMonocyte', 'AbsBasophil',
                'BilirubinTotal', 'GGT', 'PercLymphocytes', 'PercMonocytes',
                'PercBasophils', 'Phosphorus', 'PercEosinophils']

In [ ]:
for pat in tqdm(df['subject_id'].unique()): # for each subject
    df_pat = df[df['subject_id']==pat]
    for c in imputed_vars:
        if (df_pat[c].isna().sum()>0) & (df_pat[c].isna().sum()<len(df_pat)-1):
            #b, a = np.polyfit(df_pat.loc[df_pat[c].notnull(),'delta'], df_pat.loc[df_pat[c].notnull(),c], 1)
            df.loc[(df['subject_id']==pat) & (df[c].isnull()),c] = np.interp(df_pat.loc[df_pat[c].isnull(),'delta'], df_pat.loc[df_pat[c].notnull(),'delta'], df_pat.loc[df_pat[c].notnull(),c])

In [ ]:
df.loc[df['BMI'].isnull(), 'BMI'] = df.loc[df['BMI'].isnull(), 'Weight']/((df.loc[df['BMI'].isnull(), 'Height']/100)**2)

In [ ]:
df = df.reset_index(drop=True)
df.to_csv('ALSdataimputed_GL.csv')

### Train test split

In [ ]:
df = pd.read_csv('ALSdataimputed_GL.csv', index_col=[0])
patients = df['subject_id'].unique()

In [ ]:
train, test = train_test_split(patients, test_size=0.2, random_state=123)
patients_df = pd.DataFrame({'subject_id':patients,'cv':np.nan})
patients_df.loc[patients_df['subject_id'].isin(test),'cv'] = 0

In [ ]:
# cross validation
kf = KFold(n_splits=4, random_state=123, shuffle=True)
n=1
for train_index, test_index in kf.split(train):
    patients_df.loc[patients_df['subject_id'].isin(train[test_index]),'cv'] = n
    n+=1

In [ ]:
patients_df.to_csv('subjects_cv.csv')

### Nearest neighbor  
Use kNN (10) to impute features missing across all time for the subject

In [ ]:
# do knn imputation for all cv folds
for cv in range(1,5):
    # load data
    df = pd.read_csv('ALSdataimputed_GL.csv', index_col=[0])
    
    # standardize data based on training data
    df_train = df[df['subject_id'].isin(patients_df.loc[patients_df['cv']!=cv,'subject_id'])].copy()
    for col in ['Age','Sex','site_bulbar','site_limb','Onset_Delta']:
        df[col+'std'] = (df[col]-df_train[col].mean()) / df_train[col].std()
        df_train[col] = (df_train[col]-df_train[col].mean()) / df_train[col].std()
    
    # impute dynamic variables
    neigh = KNeighborsRegressor(n_neighbors=10, metric='euclidean')
    for c in imputed_vars:
        df_tmp = df_train[df_train[c].notnull()]
        neigh.fit(df_tmp[['Age','Sex','site_bulbar','site_limb','Onset_Delta']], df_tmp[c])
        df.loc[df[c].isnull(), c] = neigh.predict(df.loc[df[c].isnull(), ['Agestd','Sexstd','site_bulbarstd','site_limbstd','Onset_Deltastd']])
    
    # standardize data for static var imputation
    df_train = df[df['subject_id'].isin(patients_df.loc[patients_df['cv']!=cv,'subject_id'])].copy()
    df_train = df_train.groupby('subject_id').first().reset_index()
    df_subj = df.groupby('subject_id').first().reset_index()
    for col in ['Age','Sex','site_bulbar','site_limb']: 
        df_subj[col+'std'] = (df_subj[col]-df_train[col].mean()) / df_train[col].std()
        df_train[col] = (df_train[col]-df_train[col].mean()) / df_train[col].std()
        
    # height, dianosis_delta -> KNN regressor
    for c in ['Height','Diagnosis_Delta']:
        df_tmp = df_train[df_train[c].notnull()]
        neigh.fit(df_tmp[['Age','Sex','site_bulbar','site_limb']], df_tmp[c])
        missing_subj = df_subj[df_subj[c].isnull()]
        missing_subj[c] = neigh.predict(missing_subj[['Agestd','Sexstd','site_bulbarstd','site_limbstd']])
        df = df.merge(missing_subj[['subject_id',c]].rename({c:c+'_imputed'},axis=1), how='left', on='subject_id')
        df[c] = df[[c+'_imputed',c]].fillna(method='ffill',axis=1)[c]
        
    # riluzole -> KNN classifier
    df_tmp = df_train[df_train['RiluzoleUse'].notnull()]
    neigh = KNeighborsClassifier(n_neighbors=10, metric='euclidean')
    neigh.fit(df_tmp[['Age','Sex','site_bulbar','site_limb']], df_tmp['RiluzoleUse'])
    missing_subj = df_subj[df_subj['RiluzoleUse'].isnull()]
    missing_subj['RiluzoleUse'] = neigh.predict(missing_subj[['Agestd','Sexstd','site_bulbarstd','site_limbstd']])
    df = df.merge(missing_subj[['subject_id','RiluzoleUse']].rename({'RiluzoleUse':'RiluzoleUse_imputed'},axis=1), how='left', on='subject_id')
    df['RiluzoleUse'] = df[['RiluzoleUse_imputed','RiluzoleUse']].fillna(method='ffill',axis=1)['RiluzoleUse']
    
    # drop unwanted columns
    df = df.drop(columns=['Agestd','Sexstd','site_bulbarstd','site_limbstd','Onset_Deltastd','Height_imputed','Diagnosis_Delta_imputed','RiluzoleUse_imputed'])
    
    # recompute BMI
    df.loc[df['BMI'].isnull(), 'BMI'] = df.loc[df['BMI'].isnull(), 'Weight']/((df.loc[df['BMI'].isnull(), 'Height']/100)**2)
    
    # assign train test labels
    df['train'] = 1
    df.loc[df['subject_id'].isin(patients_df.loc[patients_df['cv']==cv,'subject_id']), 'train'] = 0
    
    # save data
    df.to_csv('ALSdataimputedknn_GL_cv' + str(cv) + '.csv')

In [ ]:
# drop unwanted columns
df = df.drop(columns=['Agestd','Sexstd','site_bulbarstd','site_limbstd','Onset_Deltastd','Height_imputed','Diagnosis_Delta_imputed','RiluzoleUse_imputed'])

In [ ]:
# recompute BMI
df.loc[df['BMI'].isnull(), 'BMI'] = df.loc[df['BMI'].isnull(), 'Weight']/((df.loc[df['BMI'].isnull(), 'Height']/100)**2)

In [ ]:
df.to_csv('ALSdataimputedknn_usingalldata.csv')

## Correct diagnosis delta imputation

In [ ]:
df_noimpute = pd.read_csv('ALSdatacleaned_noimputation.csv', index_col=[0])
subjs = df_noimpute.loc[df_noimpute['Diagnosis_Delta'].isnull(), 'subject_id'].unique()

In [ ]:
for cv in range(5):
    print(cv)
    df = pd.read_csv('ALSdataimputedknn_GL_cv'+str(cv)+'.csv', index_col=[0])
    for s in subjs:
        imputed_dd = df.loc[(df['subject_id']==s) & df['delta']==0, 'Diagnosis_Delta'].values[0]
        df.loc[df['subject_id']==s, 'Diagnosis_Delta'] = imputed_dd - df.loc[df['subject_id']==s, 'delta']
    df.to_csv('ALSdataimputedknn_GL_cv'+str(cv)+'.csv')